# Library

In [1]:
import os ,time

import glob
import random
from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torch.optim.lr_scheduler import _LRScheduler
import torch_optimizer as optim

import numpy as np
import pandas as pd, cv2
from tqdm import tqdm
from easydict import EasyDict
from sklearn.preprocessing import LabelEncoder #Label Encoder로 숫자로 변경함
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from ptflops import get_model_complexity_info

from collections import Counter
import argparse
import neptune.new as neptune
import timm

#안하면 오류남
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore')

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Load Data

In [2]:
pd.set_option('display.max_row',10) #가로 제한 수 10 밑에서 풀어버려서;
train_data = pd.read_csv("../Data/train.csv")
display(train_data)

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good
...,...,...,...,...,...
2995,2995,12995.png,screw,good,screw-good
2996,2996,12996.png,wood,good,wood-good
2997,2997,12997.png,zipper,good,zipper-good
2998,2998,12998.png,tile,oil,tile-oil


In [3]:
!cd ../Data

In [4]:
!ls

 Data_aug.ipynb			        Make_DataSet.ipynb   Predit.ipynb
'Jupyter-notebook NBextensions.ipynb'   Make_Models.ipynb    result


### Label Encoder Data

In [5]:
pd.set_option('display.max_row',None)#88개 다보기
le = LabelEncoder()
result = le.fit_transform(train_data['label'])
encoderlabel= []

for i, encoder_label in enumerate(le.classes_):
    encoderlabel.append([encoder_label,i])
encoder_dataframe = pd.DataFrame(encoderlabel, columns = ['label','encode'])
display(encoder_dataframe)

,label,encode
0,bottle-broken_large,0
1,bottle-broken_small,1
2,bottle-contamination,2
3,bottle-good,3
4,cable-bent_wire,4
5,cable-cable_swap,5
6,cable-combined,6
7,cable-cut_inner_insulation,7
8,cable-cut_outer_insulation,8
9,cable-good,9


# 데이터 증강

## 감마 변환

## 데이터 증강 ver로 선택하면 가능

In [6]:
def augmentation(img_size, ver):
    if ver==1:
        transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.Normalize([0.43303847,0.4034577, 0.39415097], [0.18344551,0.17549995, 0.1647388])
                ])   
    elif ver==2:
        transform = transforms.Compose([
                transforms.RandomRotation(degrees=20),
                transforms.RandomAffine(degrees=(10,30)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.Resize((img_size, img_size)),
                transforms.Normalize([0.43303847,0.4034577, 0.39415097], [0.18344551,0.17549995, 0.1647388])
            ])   
    elif ver==3:
        transform = transforms.Compose([
                transforms.RandomRotation(degrees=180),
                transforms.RandomAffine(degrees=(10,30)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.Resize((img_size, img_size)),
                transforms.Normalize([0.43303847,0.4034577, 0.39415097], [0.18344551,0.17549995, 0.1647388])
            ])  
    elif ver==4:
        transform = transforms.Compose([
                transforms.RandomRotation(degrees=20),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.Resize((img_size, img_size)),
                transforms.Normalize([0.43303847,0.4034577, 0.39415097], [0.18344551,0.17549995, 0.1647388])
            ])      
    return transform

# Custom DataLoader

## Train_Dataset

In [7]:
class Train_Dataset(Dataset):
    def __init__(self, df, transform=None, label_encoder=None, is_training=False):
        
        self.file_name = df['file_name'].values
        self.label = df['label'].values
        self.le = label_encoder
        self.transform = transform
        self.is_training = is_training
        self.data_path = '../Data/train data'

        print(f"데이터셋 생성 완료,,\n 데이터셋 사이즈 : {len(self.file_name)}")
        
    def __getitem__(self, index):
        img_path = os.path.join(self.data_path, self.file_name[index])
        image = cv2.imread(img_path).astype(np.float32) #numpy형식으로 불러옴
        label = self.label[index]
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)/ 255.0 #데이터 증강을 위해 바꿈
        image_transform = self.transform
            
        #image transform 시행
        image = image.transpose(2,0,1) #HWC ->> CHW
        image = image_transform(torch.from_numpy(image))
    
        return image, label
    
    def __len__(self):
        return len(self.file_name)

## TestDataset Loader

In [8]:
class Test_Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.file_name = df['file_name'].values
        self.transform = transform

        print(f"테스트 데이터셋 생성 완료,,\n 데이터셋 사이즈 : {len(self.file_name)}")

    def __getitem__(self, index):        
        image = np.array(Image.open(f'../data/test/{self.file_name[index]}').convert('RGB'))

        if self.transform is not None:
            image = self.transform(Image.fromarray(image))

        return image

    def __len__(self):
        return len(self.file_name)

## Dataset Loader

In [9]:
def get_loader(df, phase: str, batch_size, shuffle, num_workers, transform, label_encoder=None, is_training=False):

    if phase == 'test':
        dataset = Test_Dataset(df, transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
    else:
        dataset = Train_Dataset(df, transform,label_encoder=label_encoder, is_training=is_training)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True, drop_last=False)

    return data_loader

# config 설정

In [21]:
def getConfig():
    parser = argparse.ArgumentParser()

    parser.add_argument('--data_path', type=str, default='../Data')
    parser.add_argument('--file_name', type=str, default='train.csv')
    parser.add_argument('--fold', type=int, default=0, help='Validation Fold')
    parser.add_argument('--Kfold', type=int, default=5, help='Number of Split Folds')
    parser.add_argument('--tag', default='Default', type=str, help='tag')

    # Model parameter settings
    parser.add_argument('--model_type', type=str, default='', help='CNN or Transformer')
    parser.add_argument('--encoder_name', type=str, default='regnety_040') #regnety 모델사용
    parser.add_argument('--drop_path_rate', type=float, default=0.2)
    parser.add_argument('--num_classes', type=int, default=88)

    # Training parameter settings
    ## Base Parameter
    parser.add_argument('--epochs', type=int, default=50)
    parser.add_argument('--img_size', type=int, default=256)
    parser.add_argument('--batch_size', type=int, default=128)
    
    parser.add_argument('--initial_lr', type=float, default=1e-6)
    parser.add_argument('--weight_decay', type=float, default=1e-5)
    
    ## Augmentation
    parser.add_argument('--aug_ver', type=int, default=1)
    
    ### Cosine Annealing
    parser.add_argument('--warm_epoch', type=int, default=5)  # WarmUp Scheduler
    parser.add_argument('--max_lr', type=float, default=1e-4)

    parser.add_argument('--patience', type=int, default=15, help='Early Stopping')
    parser.add_argument('--use_aug', type=bool, default=False, help='augmentation for pill and zipper')

    # Hardware settings
    parser.add_argument('--logging', type=bool, default=True)
    parser.add_argument('--num_workers', type=int, default=4)
    parser.add_argument('--seed', type=int, default=42)
    args,_ = parser.parse_known_args()

    return args

# Network 생성

In [22]:
class Net(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.encoder = timm.create_model(args.encoder_name, pretrained=False, drop_path_rate = args.drop_path_rate, num_classes = args.num_classes)
        
    def forward(self, x):
        x = self.encoder(x)
        return x

# Scheduler 설정 LOSS CLASS

In [23]:
class Set_lr(_LRScheduler):
    def __init__(self, optimizer, total_iters, last_epoch=-1):
        
        self.total_iters = total_iters
        super(Set_lr,self).__init__(optimizer, last_epoch=last_epoch)

    def get_lr(self):
        return [base_lr * self.last_epoch / (self.total_iters + 1e-7) for base_lr in self.base_lrs]
    
class Loss_setting(object):
    def __init__(self, num=40):
        self.num = num
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.losses = []

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.losses.append(val)

# Train 함수 생성

In [24]:
class Trainer():
    def __init__(self, args, try_count):
        super(Trainer, self).__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        save_path = os.path.join('result/', str(try_count).zfill(3))
        os.makedirs(save_path, exist_ok=True)
        
        #neptune ai 를 사용해보았다.
        run = neptune.init(
        project="twogudak/czion",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhMDljNTU0ZC03NGM4LTQwNzEtYTBlNi0yYzE1OGZiNmE1MjkifQ==",
        tags=args.tag
        )
        
        run["parameters"] = args
        
        # Train, Valid Dataset Load
        train_data = pd.read_csv(os.path.join(args.data_path, args.file_name))
        le = LabelEncoder()
        train_data['label'] = le.fit_transform(train_data['label']) #라벨 인코딩 작업
        print(len(le.classes_))
        
        #Fold Setting
        KFold = kf = StratifiedKFold(n_splits=args.Kfold)
        for fold_num,(_,val_idx) in enumerate(KFold.split(train_data, y = train_data['label'])):
            train_data.loc[val_idx,'fold'] = fold_num
            
        val_data = train_data[train_data['fold'] == args.fold].reset_index(drop=True)
        train_data = train_data[train_data['fold'] != args.fold].reset_index(drop=True)
        
        #Data Augmentation
        self.train_transform = augmentation(img_size=args.img_size, ver=args.aug_ver)
        self.test_transform = augmentation(img_size=args.img_size, ver=1)
        
        #Load Train Data
        self.train_loader = get_loader(train_data, phase='train', batch_size=args.batch_size, shuffle=True,
                                       num_workers=args.num_workers, transform=self.train_transform,label_encoder=le, is_training=args.use_aug)
        
        self.val_loader = get_loader(val_data, phase='train', batch_size=args.batch_size, shuffle=False,
                                       num_workers=args.num_workers, transform=self.test_transform,label_encoder=le, is_training=False)
        
        #Model Load
        self.model = Net(args).to(self.device)
        amcs, params = get_model_complexity_info(self.model, (3,args.img_size, args.img_size), as_strings=True,
                                               print_per_layer_stat=False, verbose=False)
        
        print(f'Macs : {amcs}')
        print(f'params : {[params]}')
        
        #Loss
        self.criterion = FocalLoss()
        
        #Optimizer
        self.optimizer = optim.Lamb(self.model.parameters(), lr=args.initial_lr, weight_decay=args.weight_decay)
        iter_per_epoch = len(self.train_loader)
        
    
        #Scheduler
        self.warmup_scheduler = Set_lr(self.optimizer, iter_per_epoch * args.warm_epoch)
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr=args.max_lr, steps_per_epoch=iter_per_epoch, epochs=args.epochs)

        load_epoch=0
        
        #Train, Val
        
        best_loss = np.inf
        best_acc = 0
        best_f1 = 0
        best_epoch = 0
        early_stopping = 0
        start = time.time()
        for epoch in range(load_epoch+1, args.epochs+1):
            self.epoch = epoch

            if epoch > args.warm_epoch:
                self.scheduler.step()
                    
            train_loss, train_acc, train_f1 = self.training(args)
            state_dict = self.model.state_dict() #Save

            val_loss, val_acc, val_f1 = self.validate(phase='val')

            #neptune

            if args.logging == True:               
                run['Train loss'].log(train_loss)
                run['Train acc'].log(train_acc)
                run['Train f1'].log(train_f1)

                run['val loss'].log(val_loss)
                run['val acc'].log(val_acc)
                run['val f1'].log(val_f1)


            #Save Model

            if val_loss < best_loss:
                early_stopping = 0 
                best_epoch = epoch
                best_loss = val_loss
                best_acc = val_acc
                best_f1 = val_f1

                torch.save({'epoch':epoch,
                            'state_dict':state_dict,
                            'optimizer': self.optimizer.state_dict(),
                            'scheduler': self.scheduler.state_dict(),}, os.path.join(save_path,'best_model.pth'))

                print(f"SAVE: {best_epoch}epoch")

            else:
                early_stopping += 1

            if early_stopping == args.patience: #계속 발전이 없으면 중지시킴
                break

            if epoch == args.epochs:
                torch.save({'epoch':epoch,
                            'state_dict':state_dict,
                            'optimizer': self.optimizer.state_dict(),
                            'scheduler': self.scheduler.state_dict(),
                    }, os.path.join(save_path,'CZION_.pth'))
                print('---SAVE: last epoch--------')
                    
        #epoch끝
        print(f"Best Val Epoch:{best_epoch}, Val Loss : {best_loss}, Val Acc:{best_acc}, Best Val F1:{best_f1}")
        end = time.time()
        print(f"Total Process time:{(end - start) / 60:.3f}Minute")
        run["Best result"] = {
            "Best_epoch" : best_epoch,
            "Best_loss" : best_loss,
            "best_acc" : best_acc,
            "best_f1" : best_f1,
        }
        run.stop()# neptune stop
        
        
    def training(self, args):
        self.model.train()
        train_loss = Loss_setting()
        train_acc = 0
        preds_list = []
        labels_list = []

        scaler = grad_scaler.GradScaler()

        for i, (images, labels) in enumerate(tqdm(self.train_loader)):
            images = torch.tensor(images, device=self.device, dtype=torch.float32)
            labels = torch.tensor(labels, device=self.device, dtype=torch.long)

            if self.epoch <= args.warm_epoch:
                self.warmup_scheduler.step()

            self.model.zero_grad(set_to_none=True)

            with autocast():
                preds = self.model(images)
                loss = self.criterion(preds, labels)
            scaler.scale(loss).backward()
            scaler.step(self.optimizer)
            scaler.update()

            # Metric
            train_acc += (preds.argmax(dim=1) == labels).sum().item()
            preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
            labels_list.extend(labels.cpu().detach().numpy())
            train_loss.update(loss.item(), n=images.size(0))
                
                
        train_acc /= len(self.train_loader.dataset)
        train_f1 = f1_score(np.array(labels_list), np.array(preds_list), average='macro')

        print(f'Epoch:[{self.epoch}/{args.epochs}]')
        print(f'Train Loss:{train_loss.avg} | Acc:{train_acc} | F1:{train_f1}')

        return train_loss.avg, train_acc, train_f1
                

    def validate(self, phase='val'):
        self.model.eval()
        with torch.no_grad():
            val_loss = Loss_setting()
            val_acc = 0
            preds_list = []
            labels_list = []

            for images, labels in tqdm(self.val_loader):
                images = torch.tensor(images, device=self.device, dtype=torch.float32)
                labels = torch.tensor(labels, device=self.device, dtype=torch.long)

                preds = self.model(images)
                loss = self.criterion(preds, labels)

                val_acc += (preds.argmax(dim=1) == labels).sum().item()
                preds_list.extend(preds.argmax(dim=1).cpu().detach().numpy())
                labels_list.extend(labels.cpu().detach().numpy())

                val_loss.update(loss.item(), n=images.size(0))
            val_acc /= len(self.val_loader.dataset)
            val_f1 = f1_score(np.array(labels_list), np.array(preds_list), average='macro')

            print(f'{phase} Loss:{val_loss.avg} | Acc:{val_acc} | F1:{val_f1}')

        return val_loss.avg, val_acc, val_f1

In [25]:
bottle_df = pd.read_csv("../Data/bottle_df.csv")
display(bottle_df)

bottle_df["label"].unique()

,index,file_name,class,state,label
0,4,10004.png,bottle,good,bottle-good
1,26,10026.png,bottle,good,bottle-good
2,53,10053.png,bottle,contamination,bottle-contamination
3,58,10058.png,bottle,good,bottle-good
4,60,10060.png,bottle,good,bottle-good
5,94,10094.png,bottle,good,bottle-good
6,159,10159.png,bottle,good,bottle-good
7,172,10172.png,bottle,good,bottle-good
8,204,10204.png,bottle,broken_large,bottle-broken_large
9,207,10207.png,bottle,good,bottle-good


array(['bottle-good', 'bottle-contamination', 'bottle-broken_large',
       'bottle-broken_small'], dtype=object)

In [26]:

class FocalLoss(nn.Module):

    def __init__(self, gamma=5.0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
        self.ce = nn.CrossEntropyLoss(reduction="none")

    def forward(self, input, target):
        logp = self.ce(input, target)
        p = torch.exp(-logp)
        loss = (1 - p) ** self.gamma * logp
        return loss.mean()

In [28]:
# Random Seed
args = getConfig()
seed = args.seed
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = True
    
# args.batch_size = 16
# args.drop_path_rate = 0.2
# args.encoder_name = "efficientnet_b7"
# args.img_size= 384
# args.aug_ver=2
# args.num_classes= 88
# args.epochs=100
# args.warm_epoch=3
# args.fold = 0


# Trainer(args, save_path,1)

# args.batch_size = 16
# args.drop_path_rate = 0.2
# args.encoder_name = "efficientnet_b6"
# args.img_size= 384
# args.aug_ver=4
# args.num_classes= 88
# args.epochs=100
# args.warm_epoch=3
# args.fold = 1


# Trainer(args, save_path,2)

# args.batch_size = 16
# args.drop_path_rate = 0.2
# args.encoder_name = "efficientnet_b6"
# args.img_size= 384
# args.aug_ver=2
# args.num_classes= 88
# args.epochs=100
# args.warm_epoch=3
# args.fold = 2

# Trainer(args, save_path,3)

# args.batch_size = 16
# args.drop_path_rate = 0.2
# args.encoder_name = "efficientnet_b7"
# args.img_size= 384
# args.aug_ver=4
# args.num_classes= 88
# args.epochs=100
# args.warm_epoch=3
# args.fold = 3

# Trainer(args, save_path,4)



args.batch_size = 3
args.drop_path_rate = 0.2
args.encoder_name = "efficientnet_b5"
args.img_size= 512
args.aug_ver=4
args.num_classes= 4
args.epochs=50
args.warm_epoch=3
args.fold = 0
args.patience = 15
#args.data_path = "../Data/"
args.seed = 0
args.file_name = "bottle_df.csv"


Trainer(args,10)

https://app.neptune.ai/twogudak/czion/e/CZION-39
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
4
데이터셋 생성 완료,,
 데이터셋 사이즈 : 142
데이터셋 생성 완료,,
 데이터셋 사이즈 : 36
Macs : 12.28 GMac
params : ['28.35 M']


100%|███████████████████████████████████████████| 48/48 [00:36<00:00,  1.31it/s]


Epoch:[1/50]
Train Loss:0.6577219337512727 | Acc:0.7253521126760564 | F1:0.2297413793103448


100%|███████████████████████████████████████████| 12/12 [00:01<00:00,  8.13it/s]


val Loss:0.22089469153434038 | Acc:0.05555555555555555 | F1:0.02631578947368421
SAVE: 1epoch


...

100%|███████████████████████████████████████████| 48/48 [00:15<00:00,  3.07it/s]


Epoch:[19/50]
Train Loss:0.5651778017311189 | Acc:0.8450704225352113 | F1:0.2760747004933052


100%|███████████████████████████████████████████| 12/12 [00:01<00:00, 11.36it/s]

val Loss:0.2367432341828438 | Acc:0.8611111111111112 | F1:0.23134328358208955
Best Val Epoch:4, Val Loss : 0.17528079131928584, Val Acc:0.8611111111111112, Best Val F1:0.23134328358208955
Total Process time:5.704Minute
Shutting down background jobs, please wait a moment...


Done!
Waiting for the remaining 26 operations to synchronize with Neptune. Do not kill this process.
All 26 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/twogudak/czion/e/CZION-39
